In [ ]:
import itertools as it

import gfapy
import holoviews as hv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
from pyarrow import csv
from tqdm.auto import tqdm

In [ ]:
hv.extension("bokeh")

In [ ]:
import re
from collections import Counter

In [ ]:
def segment_frequences(table, segment_names):
    rows = []
    for path in table.column("path"):
        read_segments = re.split(r">|<", str(path))
        segment_counts = Counter(read_segments)
        rows.append([segment_counts[s] for s in segment_names])
    return pd.DataFrame(rows, columns=segment_names, index=table.column("name"))

In [ ]:
# read1: id
# read2: id if duplex, null otherwise

In [ ]:
# gaf row -> row (columns: segments, values: cigar strings with local coördinates [duplicating cigar if necessary, null if segment does not appear)
# filter by duplex vs simplex
# filter out full length from partial-length and unexpected topology (e.g., repeated segments)
# for each segment:
# - number of times it appears in path
# - localized CIGAR (suitable for aggregation, optionally weighted by read quality??)
# get quality score from read???

# NM: edit distance to reference
# AS: alignment score
# dv: ?
# id: ?

In [ ]:
gfa = gfapy.Gfa.from_file("nao745bc.gfa")

In [ ]:
!du -hs *.gaf

In [ ]:
# gaf_filename = "duplex_hac1_subsample_dbg.gaf"
# gaf_filename = "duplex_hac1_subsample_dbg2.gaf"
# gaf_filename = "duplex_hac1_subsample_vg2.gaf"
gaf_filename = "duplex_sup1_vg2.gaf"

In [ ]:
%%time
# PATH_REGEX = r"([><][^\s><]+(:\d+-\d+)?)+|([^\s><]+)"
# SEE: http://samtools.github.io/hts-specs/SAMv1.pdf
# and https://samtools.github.io/hts-specs/SAMtags.pdf
SAM_TAG_TYPES = {
    "A": pa.dictionary(pa.int32(), pa.string()),
    "f": pa.float32(),
    "i": pa.int32(),
    "Z": pa.string(),
}
with open(gaf_filename, "r") as f:
    first = f.readline().split("\t")
tags = first[12:]
tag_column_types = {(t := tag.split(":"))[0]: SAM_TAG_TYPES[t[1]] for tag in tags}
column_types = {
    "name": pa.string(),
    "query_length": pa.uint64(),
    "query_start": pa.uint64(),
    "query_end": pa.uint64(),
    "strand": pa.dictionary(pa.int32(), pa.string()),
    "path": pa.string(),
    "path_length": pa.uint64(),
    "path_start": pa.uint64(),
    "path_end": pa.uint64(),
    "residue_matches": pa.uint64(),
    "block_length": pa.uint64(),
    "mapping_quality": pa.uint8(),
    **{tag: pa.string() for tag in tag_column_types.keys()},
}
read_options = csv.ReadOptions(column_names=column_types.keys())
parse_options = csv.ParseOptions(delimiter="\t")
convert_options = csv.ConvertOptions(column_types=column_types)
with csv.open_csv(
    gaf_filename,
    read_options=read_options,
    parse_options=parse_options,
    convert_options=convert_options,
) as f:
    # tt = f.read_next_batch()
    segment_counts = Counter()
    barcode_counts = Counter()
    total = 0
    duplex = 0
    singleton = 0
    filtered = 0
    # while True:
    # for _ in tqdm(it.islice(it.count(), 10)):
    for _ in tqdm(it.count()):
        try:
            table = f.read_next_batch()
        except StopIteration:
            break
        # break
        freqs = segment_frequences(table, gfa.segment_names)
        duplex += freqs.index.str.contains(";").sum()
        # filtered_freqs = freqs[(freqs.max(axis=1) == 1) & ((freqs["BIT0OFF"] == 1) | (freqs["BIT0ON"] == 1)) & (freqs["pPhlF"] == 1)]
        filtered_freqs = freqs[
            # freqs.index.str.contains(";")
            (freqs.max(axis=1) == 1)
            & ((freqs["BIT0OFF"] == 1) | (freqs["BIT0ON"] == 1))
            & (freqs["RBS1"] == 1)
        ]
        # filtered_freqs = freqs
        filtered += len(filtered_freqs)
        total += len(freqs)
        segment_counts.update(list(filtered_freqs.itertuples(index=False)))
        barcode_counts.update(
            list(
                filtered_freqs.loc[
                    :, filtered_freqs.columns.str.startswith("BIT")
                ].itertuples(index=False)
            )
        )

In [ ]:
(filtered, total, filtered / total, duplex, duplex / total)

In [ ]:
def bit_sums(freqs):
    return pd.DataFrame(
        {f"BIT{i}": freqs[f"BIT{i}ON"] + freqs[f"BIT{i}OFF"] for i in range(30)}
    )

In [ ]:
filtered_freqs.mean(axis=0)

In [ ]:
filtered_freqs.loc[:, filtered_freqs.columns.str.endswith("ON")].mean(axis=0).plot.bar()

In [ ]:
filtered_freqs.mean(axis=0).plot.bar()

In [ ]:
segment_counts.most_common(3)

In [ ]:
barcode_counts.most_common(3)

In [ ]:
plt.hist(barcode_counts.values(), bins=100, log=True);

In [ ]:
n, bins, patches = plt.hist(
    barcode_counts.values(),
    100,
    histtype="step",
    density=False,
    cumulative=-1,
    log=True,
)

In [ ]:
n, bins, patches = plt.hist(
    barcode_counts.values(),
    100,
    range=(0, 10),
    histtype="step",
    density=False,
    cumulative=-1,
    log=True,
)

In [ ]:
len(barcode_counts)

In [ ]:
sum(1 for v in barcode_counts.values() if v == 1)

In [ ]:
sum(1 for v in barcode_counts.values() if v == 2)

In [ ]:
sum(1 for v in barcode_counts.values() if v == 3)

In [ ]:
sum(1 for v in barcode_counts.values() if v == 4)

In [ ]:
sum(1 for v in barcode_counts.values() if v >= 5)

In [ ]:
sum(1 for v in barcode_counts.values() if 5 <= v < 20)

In [ ]:
sum(1 for v in barcode_counts.values() if 20 <= v < 100)

In [ ]:
sum(1 for v in barcode_counts.values() if v >= 100)

In [ ]:
sum(1 for v in barcode_counts.values() if v >= 100)

In [ ]:
sum(1 for v in barcode_counts.values() if v >= 2)

In [ ]:
max(barcode_counts.values())

In [ ]:
sum(v for v in barcode_counts.values() if v == 1) / total

In [ ]:
sum(1 for v in barcode_counts.values() if v >= 2)

In [ ]:
sum(v for v in barcode_counts.values() if v >= 10)

In [ ]:
sum(v for v in barcode_counts.values() if v >= 10) / filtered

In [ ]:
sum(v for v in barcode_counts.values() if v >= 50) / filtered

In [ ]:
sum(v for v in barcode_counts.values() if v >= 100) / filtered

In [ ]:
filtered

In [ ]:
sum(v for v in barcode_counts.values())

In [ ]:
n, bins, patches = plt.hist(
    barcode_counts.values(),
    100,
    range=(0, 20),
    histtype="step",
    density=True,
    cumulative=-1,
    log=True,
)

In [ ]:
f = extract_barcodes(tt, gfa.segment_names)

In [ ]:
tuple(next(f.iterrows())[1].values)

In [ ]:
f.max(axis=1)

In [ ]:
tt.column("path")

In [ ]:
re.split(r">|<", s)

In [ ]:
paths = tt.select(["path"]).to_pandas()

In [ ]:
s = paths.iloc[0, 0]

In [ ]:
PATH_REGEX

In [ ]:
m = re.match(PATH_REGEX, paths.iloc[0, 0])

In [ ]:
m.groups()

In [ ]:
tt.take([0]).to_pandas()

In [ ]:
gfa.try_get_segment("BIT1OFF")

In [ ]:
tt2 = tt.to_pandas()

In [ ]:
tt2["path_length"].plot.hist(bins=50)

In [ ]:
tt2["mapping_quality"].value_counts()

In [ ]:
t = table.to_pandas()

In [ ]:
import uuid

In [ ]:
u = uuid.UUID("6e507a8a-c271-4561-8768-0f9bf9d4c301")

In [ ]:
import sys

In [ ]:
sys.getsizeof(u.int)

In [ ]:
sys.getsizeof("6e507a8a-c271-4561-8768-0f9bf9d4c301")

In [ ]:
u.int

In [ ]:
t["cg"][0]

In [ ]:
t["name"].str.split(";")

In [ ]:
# 1) segment_cigars df (segment coördinates, normalize orientation) [numba]
# 2) segment_mismatches df (cellwise apply, get insertions/deletions/mismatches/equal)
# 3) filter on barcode mismatches (?)
# 4) group segment_cigars by barcode, run cigar_aggregation on non-barcode (or all!) segments
# 5)

In [ ]:
import stream
import vg_pb2

In [ ]:
gam_filename = "duplex_sup1_subsample_vg2.gam"

In [ ]:
from itertools import islice

In [ ]:
len(
    "CACTGGAAACATCAAGGTCGACGAGGAGAGCGCGGGTGAGAGGGATTCGTTACCAATAGAGCAACGTACGACGTTCAATATAATGCTAGCTACTAGCGCTGTCTGTACTTGTATCAGTACACTGACCAGTCCCTAAAGAACGAAACAACCGCCTCTACAAATAATTTTGTTTAACCATAACAAAGAAAGGGGGTATTCTAATGAAACCAGTAACGTTATACGATGTCGCAGAGTATGCCGGTGTCTTCTTATATGACCGTTTCCCGCGTGGTGAACCAGGCCAGCCACGTTCTGCGAAAACGCGGAAAAAGTGGAAGCGGCGATGGTGGAGCTGAATTACATTCCCGACCGCGTGGCACAACAACTGGCGGGCAAACAGTCGTTGCTGATTGGCGTTGCCACCTCCAGTCTGGCCCTGCACGCGCCGTCGCAAATTGTCGCGGCGATTAAATTCGCGCCGATCAACTGGGTGCGCCAGCGTGGTGGTCTGAATCGGTAGAACGAAGCGGCGTCGAAGCCTGTAAAGCGGCGGTGCACAATCTTCTCGCGCAACGCGTCAGGTGGGCTGATCATTAACTATCCGCTGGATGACCAGGATGCCATTGCTGTGGAAGCTGCCTGCACTAATGTTCCGGCGTTATTTCTTGATGTTCTGACCAGACACCCATCAACAGTATTATTTACTCCCATGAGGACGGTACGCGACTGGGCGTGGAGCATCTGGTCGCATTGGGTCACCAGCAAATCGCGCTTGTTAGCGGGCCCATTAAGTTCTGTCTCGGCGCGTCTGCGTCTGGCTGGCTGGCATAAATATCTCACTCGCAATCAAATTCAGCCGATAGCGGAACGGGAAGGCGACTGGAGTGCCATGTCGGTTTTCAACAAACCATGCAAATGCTGAATGAGGGCATCGTTCCCACTGCGATGCTGGTTGCCAACGATCAGATGGCGCTGGGCGCAATGCGCGCCATTACCGAGTCCGGGCTGCGCGTTGGTGCGGATATCTCGGTAGTGGTAGGATACGACGATACCGAAGATAGCTCATGTTAATCCCGCCGTTAACCACCATCAACAGGATTTTTCGCCTGCTGGGGCAAACCAGCGTGGACCGCTTGCTGCAACTCTCAGGCCAGGCGGTGAAGGGCAATCAGCTGTTGCCAGTCTCACTGGTGAAAAGAAAAACCACCCTGGCGCCCAATACGCAAACCGCCTCTCCCCGCGCGTTGGCCGATTCATTAATGCAGCTGGCACGACAGGTTTCCCGACTGGAAAGCAGGCAGTAATAAAGGTCCGGCAATTAAAAAAGCGGCTAACCACGCCGCTTTTTTTAGATGTCACTCGGTACCAAATTCCAGAAAGAGGCCTCCCGAAAGGGGGGCCTTTTTTCGTTTTTGGTCCGCTTTGAAGGAGTGTTGACAATTAATCATCGGCTCGATAATGTGTGGAATTGTGAGCGCTCACAATTTACTAGCGCTGTCTGTACTTGTATCAGTACACTGACGAGTCCCTAAAGGACGAAACACCGCCTCTACAAATAATTTTGTTCAACCATAACAAAGAAAGGGGGTCCTGTAATGGCACGTACCCGAGCCGTAGCAGCATTGGTAGCCTGCGTAGTCCGCATACCCATAAAGCAATTCTCGACCAGCACCATTGAAATCCTGAAAAGAATGTGGTTATAGCGGTCTGAGCATTGAAAGCGTGGCACGTCGCGCCGGTGCAGGCAAACCGACCATTCATCGTTGGTGGACCAACAAAGCAGCACTGATTGCCGAAGTGTATGAATGAAATCGAACAGGTACGTAAATTTCCGGATTTGGGTAGCTTTAAAGCCGATCTGGATTTTCTGCTGCATAATCTGTGGAAAGTTTGGCGTGAAACCATTTGTGGTGAAGCATTTCGTTGTGTTATTGCAGAAGCACAGTTGGACCCTGCAACCCTGACCCAACTGAAAGATCAGTTTATGGAACGTCGTCGTGAGATACCGAAAAAACTGGTTGAAGATGCCATTAGCAATGGTGAACTGCCGAAAGATATCAATCGTGAACTGCTGCTGGATATGATTTTTGGTTTTTGGTTGGTATCGCCTGCTGACCGAACAGTTGACCGTTGAACAGGATCGTAAAGATTGACCTTCCTGCTGATTAATGGTGTTTGTCCGGGTACACAGCGTCAATAAGGTCCGGCTTATCGGTCAGTTTCACCTGATTTACGTAAAAACCCGCTTCGGCGGGTTTTTGCTTTTGAGGGGCAGAAAGATGAATGACTGTCCACGACGCTATACCCAAAAGAAAAAAAACCCCGCCCGTGACAGGGCGGGTTTTTTTTTGCTTAAATAGGAGCGACGTACGGTGGAATCTGATTCGTTACCAATTGACATGATACGGAACGTACCGTATCGTTAAGGTTACTAGCGCTGTCTGGTACTTGTATCAGTACACTGACGAGTCCCTAAAGGACGAAACACCGCCTCTACAAATAATTTTGTTTAACCATAACAGAGAAAGGGGGTTTTCTGAATGCCGAAACTGGGTATGCAGAGCATTCGTCGTCGTCAGCTGATTGATGCAACCCTGGAAGCAATTAATGAAGTTGGTATGCATGATGCAACCATTGCACAGATTGCACGTCGTGCCGGTGTTAGCACCGTATGATTAGCCATTATTTCCGCGATAAACGGTCGACTGGATGGAAGCAACGATGCGTGATATTACCAGCCAGCTGCGTGATGCAGTTCTGAATCGTCTGCATGCACTGCCGCAGGGTAGCGCAGAACAGCGTCTGCAGGCAATTGTTGGTGGTAATTTTGATGAAACCCAGGTTAGCAGCGCAGCAATGAAAGCATGGCTGGCATTTGGGCAATCGAGCATGCATCAGCCGATGCTGTATCGTCTGCAGCAGGTTAGCAGTCGTCGTCTGCTGAGCAATCTGGTTAGCGAATTTCGTCCGTGAACTGCCTCGTGAACAGGCACAAGAGGCAGGTTATGGTCTGGCAGCACTGATTGATGGTCTGTGGCTGCGTGCAGCACTGAAGCGGTAAACCGCTCGGATAAAACCCGTGCAAATAAGCCTGACCCGTCATTTTATCACCCAGCATCTGCCGACCGATTAATAAGGTCTTTAAAAAGAAACCTCCGCATTGCGGAGGTTTCGCCTTTTGATACTCTGTCTGAAGTAATTCTTGCCGCAGTGAAAAATGGTGCCCATCGGCGCCATTTTTTTATGCTTCCATTAGAAAGCAAAAAGCCTGCTAGAAAGCAGGCTTTTAATTTGGCTCCTCGGGCACGCTTACTGAGGAGCGAGGATAAGATAAAATTACCACAACTGCGAGCCCTTCCACCAAAAAAAACAAGATAGCCGCGCGAACGCGGCTAACTGTTGAAAAAAACAGATAACAGATACGAAGAATCTGTTATCGTAAAAACCCCTCAAAGACCGTTTAAGAAGGCCCAAGGGGTTATTACTGATGGCAATGTGATGACCTCATCATTCGCTCATCATCACCAACTGAATCTCACCTTCCACTTCACGATCCATATCCTTCTCACCCTGACCTTTCTCCATACCCAACTTTCCTAACAACCAACTAACCTGCCTCTTCATCAGTCCATGATTCTCCCTCTATCAGCTCCAGCAACCACACTCATAATGCATATCCAACCACAACCTCAGTCAACTCATTACCCACAACCGCCCTTACCAACAACAATCCTGATTCCATACCCACTCCGTATCCTTCAATCCCTCCACAGCATAACCCTACACACAACACGTATCGTTATGTAATTGCTAGACCCTTTACAAACACACCTGAGGTAAACCCATCCCACATCCTGTCAATCCACCATTCCTCAACGAAACTTCATCACTCTCCTCCTCACTAATTCACAACTCTCGTCCCCCACTTCACTTCACTATGCTTCTTCTCTTACACCCTCTGTATCATCCTCCTTCTCTCACCACCATAACTCCATCCTTCGCCCTCTTACTTATCTGACCCAGACCAACTTCCACACTCACTGGATCTCATCAATCCCA"
)

In [ ]:
!head -n 4 duplex_sup1_subsample.fastq

In [ ]:
for msg in islice(stream.parse(gam_filename, vg_pb2.Alignment), 2):
    print(msg)
    print()
    print("************")
    print()

In [ ]:
for msg in stream.parse(gam_filename, vg_pb2.Alignment):
    if ";" in msg.name:
        continue
    print(msg)
    print()
    print("************")
    print()
    break

In [ ]:
msg.name

In [ ]:
msg.quality